## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.30"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Insert normalized trajectory to db

In [ ]:
import importlib
import Python_scripts.Feature_functions.smooth_random_walk

importlib.reload(Python_scripts.Feature_functions.smooth_random_walk)

from Python_scripts.Feature_functions.smooth_random_walk import smooth_walk_sparse_band_only


In [ ]:
import pandas as pd
from tqdm import tqdm

# Step 1: Get ToyLight saline IDs with long trial durations
ToyLight_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin'"
df = pd.read_sql_query(ToyLight_q, conn)
ToyLight_id = df['id'].to_list()

# Step 2: Overwrite head_x_norm and head_y_norm
for trial_id in tqdm(ToyLight_id):
    traj = smooth_walk_sparse_band_only(
        trial_id=trial_id, conn=conn, 
        r_min=0.41, r_max=0.53, seed=trial_id)
    head_x = traj[:, 0].tolist()
    head_y = traj[:, 1].tolist()

    cursor.execute("""
        UPDATE dlc_table_temp
        SET head_x_norm = %s, head_y_norm = %s
        WHERE id = %s
    """, (head_x, head_y, trial_id))

conn.commit()
print("Simulated head_x_norm and head_y_norm written to dlc_table_temp.")


## Fetch id list

In [ ]:
ToyLight_saline_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='saline';"
df = pd.read_sql_query(ToyLight_saline_q, conn)
ToyLight_saline_id = df['id'].to_list()

ToyLight_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin';"
df = pd.read_sql_query(ToyLight_ghrelin_q, conn)
ToyLight_ghrelin_id = df['id'].to_list()

ToyLight_saline_id

## Gaussian fit position projection on diagonal

In [ ]:
import importlib
import Python_scripts.Feature_functions.diagonal_density
import Python_scripts.Feature_functions.compute_gmm_widths

importlib.reload(Python_scripts.Feature_functions.diagonal_density)
importlib.reload(Python_scripts.Feature_functions.compute_gmm_widths)

from Python_scripts.Feature_functions.diagonal_density import (
    project_onto_diagonal, 
    plot_projection_and_analyze, 
    fit_and_plot_gmm_overlay
)

from Python_scripts.Feature_functions.compute_gmm_widths import compute_gmm_widths

In [ ]:
proj = project_onto_diagonal(id=104, conn=conn)
gmm_result = fit_and_plot_gmm_overlay(proj, result=None)  # GMM

print(gmm_result)


In [ ]:
# Run separately
saline_widths = compute_gmm_widths(ToyLight_saline_id, conn)
ghrelin_widths = compute_gmm_widths(ToyLight_ghrelin_id, conn)

# Optional: compare using boxplot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({
    'width': saline_widths + ghrelin_widths,
    'group': ['Saline'] * len(saline_widths) + ['Ghrelin'] * len(ghrelin_widths)
})

plt.figure(figsize=(5, 4))
sns.boxplot(data=df, x='group', y='width')
sns.stripplot(data=df, x='group', y='width', color='black', alpha=0.5, jitter=True)
plt.ylabel('GMM width')
plt.title('Diagonal Separation (GMM width)')
plt.grid(True)
plt.show()


In [ ]:
from scipy.stats import ranksums
stat, pval = ranksums(saline_widths, ghrelin_widths)
print(f"RS test: p = {pval:.3f}")
